In [1]:
import os
import shutil
from glob import glob
import warnings

import earthpy as et
import earthpy.earthexplorer as etee
import earthpy.spatial as es
import geopandas as gpd
import geoviews as gv
from holoviews import opts
import holoviews as hv
import hvplot.pandas
import hvplot.xarray
import numpy as np
import pandas as pd 
import pyogrio
import rioxarray as rxr
import rioxarray.merge as rxrmerge

data_dir= os.path.join(et.io.HOME, et.io.DATA_NAME)
den_dir = os.path.join(data_dir, 'denver-neighborhoods')
ndvi_dir = os.path.join(data_dir, 'denver-green-space', 'processed')

for a_dir in [den_dir, ndvi_dir]:
        if not os.path.exists(a_dir):
                os.makedirs(a_dir)


#warnings.simplefilter('ignore')

In [2]:
den_path = os.path.join(den_dir, 'denver-neighborhoods.shp')
if not os.path.exists(den_path):
    den_url = ("https://www.denvergov.org/media/gis/DataCatalog/statistical_neighborhoods/shape/statistical_neighborhoods.zip")
    gpd.read_file(den_url).to_file(den_path)

gpd.read_file(den_path)
den_gdf = gpd.read_file(den_path).set_index('NBHD_NAME')

neigh_gdf = (den_gdf.loc[['Sloan Lake', 'City Park', 'West Colfax','Hampden South']])
neigh_gdf

,NBHD_ID,TYPOLOGY,NOTES,geometry
NBHD_NAME,,,,
Sloan Lake,56,None,None,"POLYGON ((-105.02523 39.75848, -105.02524 39.7..."
City Park,14,None,None,"POLYGON ((-104.94062 39.75104, -104.94063 39.7..."
West Colfax,74,None,Removed NEST Status May 2023,"POLYGON ((-105.02521 39.74636, -105.02521 39.7..."
Hampden South,33,None,None,"POLYGON ((-104.90205 39.62443, -104.90237 39.6..."


In [3]:
gpd.read_file(den_path)

,NBHD_ID,NBHD_NAME,TYPOLOGY,NOTES,geometry
0,56,Sloan Lake,None,None,"POLYGON ((-105.02523 39.75848, -105.02524 39.7..."
1,39,Jefferson Park,None,None,"POLYGON ((-105.00949 39.75393, -105.00976 39.7..."
2,14,City Park,None,None,"POLYGON ((-104.94062 39.75104, -104.94063 39.7..."
3,42,Lowry Field,None,None,"POLYGON ((-104.88470 39.73285, -104.88471 39.7..."
4,27,Fort Logan,None,None,"POLYGON ((-105.03462 39.65328, -105.03463 39.6..."
...,...,...,...,...,...
73,74,West Colfax,None,Removed NEST Status May 2023,"POLYGON ((-105.02521 39.74636, -105.02521 39.7..."
74,33,Hampden South,None,None,"POLYGON ((-104.90205 39.62443, -104.90237 39.6..."
75,67,Valverde,NEST,None,"POLYGON ((-105.01577 39.72550, -105.01582 39.7..."
76,5,Barnum West,NEST,None,"POLYGON ((-105.03945 39.72571, -105.03945 39.7..."


In [4]:
(gv.tile_sources.OSM 
 * neigh_gdf.to_crs(3857).hvplot(
     line_color = 'red',fill_color = None, 
     line_width = 3, xaxis = None, yaxis = None))

:Overlay
   .WMTS.I     :WMTS   [Longitude,Latitude]
   .Polygons.I :Polygons   [x,y]

In [10]:
def download_neighborhood_data(name, geometry, start, end):
    """
    Download NAIP raster for a given geometry, start date, and end date
    Downloads data from the National Agricultural Imagery Program (NAIP)
    given a spatial and temporal extent.

    Parameters
    ==========
    name : str
      The name used to label the download
    geometry : shapely.POLYGON
      The geometry to derive the download extent from. 
      Must have a `.bounds` attribute.
    start : str
      The start date as 'YYYY-MM-DD'
    end : str
      The end date as 'YYYY-MM-DD'

    Returns
    =======
    downloader : earthpy.earthexplorer.EarthExplorerDownloader
      Object with information about the download, including the data 
      directory.
    """

    print(f'\nNeighborhood name: {name}')
    bbox = etee.BBox(*geometry.bounds)
    naip_downloader = etee.EarthExplorerDownloader(
        dataset="NAIP", 
        label=name.lower().replace(' ','-'), 
        bbox=bbox,
        start= start, 
        end= end ,
        store_credential=True)
    naip_downloader.submit_download_request()
    naip_downloader.download(override=True)
    return naip_downloader

ndvi_stats_path = os.path.join(ndvi_dir, 'neighborhood-ndvi-stats.csv')
if os.path.exists(ndvi_stats_path):
  print('reading in NDVI stats file ...')
  ndvi_stats_df = pd.read_csv(ndvi_stats_path, index_col='neighborhood')
else:
   print('NDVI stats file DNE')
   ndvi_stats_df = pd.DataFrame()
   
for neighborhood_name, details in neigh_gdf.iterrows():
    if neighborhood_name in ndvi_stats_df.index:
      print('neighborhood stats have already been calculated. skipping')
      continue
    downloader = download_neighborhood_data(
       neighborhood_name, details.geometry,'2011-01-01','2011-12-31')

NDVI stats file DNE

Neighborhood name: Sloan Lake
Login Successful.
Searching datasets...
Using dataset alias: naip
Searching scenes...
Found 2 scenes
4 products found.
Downloads staging...

 6 downloads queued but not yet available. Waiting for 30 seconds.


 6 downloads queued but not yet available. Waiting for 30 seconds.



KeyboardInterrupt: 